<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задача:" data-toc-modified-id="Задача:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задача:</a></span></li><li><span><a href="#Решение" data-toc-modified-id="Решение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Решение</a></span></li></ul></div>

### Задача:
Дано 2 датафрейма: df1, df2

Формат таблиц: 

df1 – столбцы [‘year’, ‘month’, ‘day’, ‘Sales_PV’]

df2  - столбцы [‘Year’, ‘Month’, ‘Day’, ‘registrations’] 

Надо объединить исходные 2 датафрейма и посчитать суммарные/среднедневные продажи и регистрации за месяц.
Итоговый результат со всеми посчитанными метриками должен быть представлен в одной таблице

Напишите код, который позволит обработать исходные таблицы и предоставить результат в нужном формате. Используйте библиотеку Pandas

### Решение

In [1]:
# Испортируем библиотеки
import pandas as pd
import datetime

In [2]:
# читаем эксель-файлы с данными
df1 = pd.read_excel('df1.xlsx')
df2 = pd.read_excel('df2.xlsx')

In [3]:
# переводим заголовки колонок в нижний регистр, для удодства объединения

df1.rename(columns=str.lower, inplace=True)
df2.rename(columns=str.lower, inplace=True)
df1

,year,month,day,sales_pv
0,2024,1,1,9124
1,2024,1,2,4337
2,2024,1,3,8367
3,2024,1,4,5843
4,2024,1,5,6105
...,...,...,...,...
147,2024,5,27,5079
148,2024,5,28,4969
149,2024,5,29,9480
150,2024,5,30,9192


In [4]:
df2

,year,month,day,registrations
0,2024,1,1,88
1,2024,1,2,2
2,2024,1,3,6
3,2024,1,4,92
4,2024,1,5,78
...,...,...,...,...
147,2024,5,27,63
148,2024,5,28,54
149,2024,5,29,79
150,2024,5,30,51


In [5]:
# объединяем датафреймы по дате; удалим дубли столбцов года, месяца и дня
union_data = df1.merge(
    df2,
    on=['year','month','day'],
    how='inner'
)

# считаем суммарные продажи и регистрации за месяц
union_data['sum_month_sales'] = union_data.groupby('month')['sales_pv'].transform('sum')
union_data['sum_month_registrations'] = union_data.groupby('month')['registrations'].transform('sum')

# считаем среднедневные продажи и регистрации по каждому месяцу
union_data['mean_day_sales'] = round(union_data['sum_month_sales']/union_data.groupby('month')['day'].transform('count'),2)
union_data['mean_day_registrations'] = round(union_data['sum_month_registrations']/union_data.groupby('month')['day'].transform('count'),0)

union_data

,year,month,day,sales_pv,registrations,sum_month_sales,sum_month_registrations,mean_day_sales,mean_day_registrations
0,2024,1,1,9124,88,143480,1729,4628.39,56.0
1,2024,1,2,4337,2,143480,1729,4628.39,56.0
2,2024,1,3,8367,6,143480,1729,4628.39,56.0
3,2024,1,4,5843,92,143480,1729,4628.39,56.0
4,2024,1,5,6105,78,143480,1729,4628.39,56.0
...,...,...,...,...,...,...,...,...,...
147,2024,5,27,5079,63,159521,1485,5145.84,48.0
148,2024,5,28,4969,54,159521,1485,5145.84,48.0
149,2024,5,29,9480,79,159521,1485,5145.84,48.0
150,2024,5,30,9192,51,159521,1485,5145.84,48.0


In [6]:
# сформируем выжимку данных по каждому месяцу: удалим подробности по каждому дню, оставим только агрегированные данные 
summary = union_data.pivot_table(index = ['year','month','sum_month_sales','sum_month_registrations', 'mean_day_sales','mean_day_registrations']).reset_index()
summary = summary.drop(['day','sales_pv','registrations'], axis=1)
summary

,year,month,sum_month_sales,sum_month_registrations,mean_day_sales,mean_day_registrations
0,2024,1,143480,1729,4628.39,56.0
1,2024,2,133453,1410,4601.83,49.0
2,2024,3,136257,1853,4395.39,60.0
3,2024,4,163552,1595,5451.73,53.0
4,2024,5,159521,1485,5145.84,48.0


In [7]:
# при необходимости выгрузим данные в эксель, чтобы передать заказчику
union_data.to_excel('Полные данные.xlsx', index=False)
summary.to_excel('Саммари.xlsx', index=False)